<a href="https://colab.research.google.com/github/RaghadQadah/Automatic-Spam-Review-Detection/blob/main/SpamReviewDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**import** **Library**

In [181]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**functions to extract features from text**

In [182]:
def clean(df, text, name):
    df[name] = df[text].str.lower()
    df[name] = df[name].str.replace('http\S+|www.\S+', '', case=False)
    df[name] = df[name].str.replace('[^\w\s]', '')
    df[name] = df[name].str.replace('\d+', '')
    df[name] = df[name].apply(lambda x: ' '.join([word for word in  str(x).split() if word not in (stop)]))
    df[name] = df[name].apply(lambda x: " ".join(x for x in  str(x).split() if len(x) > 3))
    return df
def word_count(df, text, name):
    df[name] = df[text].apply(lambda x: len(str(x).split(" ")))
    return df

def hashtags_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('#')]))
    return df

def user_mentions_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('@')]))
    return df

def numerics_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.isdigit()]))
    return df

def links_count(df, text, name):
    df[name] = df[text].apply(
        lambda x: len([x for x in  str(x).split() if x.startswith(('httP', 'HTTPS', 'https', 'www', 'WWW', 'HTTP', 'http')) ]))
    return df

**Read From CSV File** 



In [183]:
df = pd.read_csv("drive/My Drive/Output.csv",encoding='latin1',low_memory=False)
df=df[:20000]

# clean review Content
clean(df, "reviewContent", "cleanContent")

,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,Filtered,cleanContent
0,---2eJpQ2HAnRQRp2v7XLA,-ORdirlxcallT7dzlZlW2A,I came to Chicago on business and was initiall...,5,5,9/28/2011,-ORdirlxcallT7dzlZlW2A,0.0,2.0,0.0,came chicago business initially supposed stay ...
1,--0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268.0,543.0,0.0,still love place took menu deeply saddened sho...
2,--1PwTt0AfyPG1pE7bB6Nw,pW9LvtC-hZ0PRAOUn7aZ2A,Alan is the best CPA that I know. He knows all...,5,0,8/15/2006,pW9LvtC-hZ0PRAOUn7aZ2A,834.0,1463.0,0.0,alan best know knows outs loop holes smart whi...
3,--87QVdPvgbDK3GeVxFRzg,92rtlWxS2aOngzkKhi-OxQ,My family has been using Korean Air for many m...,4,0,7/28/2012,92rtlWxS2aOngzkKhi-OxQ,43.0,158.0,0.0,family using korean many many years going asia...
4,--99ZJoVHP32iSd4i4nlZw,Egu4qSFR56EhGuTHZTaVjA,I hadn't been to Grisanti's since a pre-homeco...,3,0,1/28/2010,Egu4qSFR56EhGuTHZTaVjA,133.0,725.0,0.0,hadnt grisantis since prehomecoming dinner hig...
...,...,...,...,...,...,...,...,...,...,...,...
19995,A18YXCahOIwmCJ1J8derdA,KYtrBL_Cp1CudlGC1XiiSA,I have to say that one of the best attributes ...,4,0,4/24/2010,KYtrBL_Cp1CudlGC1XiiSA,174.0,1022.0,0.0,best attributes place consistency around long ...
19996,a1a-7V09z2AfspiQeBiBCw,5Y5KbsI5buMcNh2hTFaRpA,Willowcreek is a decent option for vegetarians...,3,1,7/7/2011,5Y5KbsI5buMcNh2hTFaRpA,94.0,580.0,0.0,willowcreek decent option vegetariansvegans ha...
19997,a1AIpxn2OXX48s9RTmVr7g,wwNs_kMiVEn8V61uqP3ZzA,Rate location security and staff were better t...,4,3,7/18/2008,wwNs_kMiVEn8V61uqP3ZzA,5.0,12.0,0.0,rate location security staff better vast major...
19998,a1AoCyA9VE37XPa3eF5aQA,oClGUimXsTxxh4DvuKu26Q,this is the place I shop when in MT,4,0,7/31/2011,oClGUimXsTxxh4DvuKu26Q,21.0,105.0,0.0,place shop


**TF-IDF**

In [184]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
tfidf_matrix = tfidfconverter.fit_transform(df["cleanContent"])

In [185]:
print(len(tfidf_matrix.toarray()))


20000


**calculateing cosine similarity**

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosineSim=[]
for i in range(len(tfidf_matrix.toarray())):
  sim=(cosine_similarity(tfidf_matrix[i], tfidf_matrix)).mean()
  cosineSim.append(sim)

**text processing**

In [186]:
list_of_rows = [list(row) for row in df.values]
#print(len(cosineSim))
links_count(df,"reviewContent","linksCount")
hashtags_count(df, "reviewContent", "hashtagsCount")
user_mentions_count(df, "reviewContent", "userMentionsCount")
numerics_count(df,"reviewContent","numericsCount")
clean(df, "reviewContent", "cleanContent")

word_count(df, "reviewContent", "wCount")
word_count(df, "cleanContent", "wordCount")



,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,Filtered,cleanContent,linksCount,hashtagsCount,userMentionsCount,numericsCount,wCount,wordCount
0,---2eJpQ2HAnRQRp2v7XLA,-ORdirlxcallT7dzlZlW2A,I came to Chicago on business and was initiall...,5,5,9/28/2011,-ORdirlxcallT7dzlZlW2A,0.0,2.0,0.0,came chicago business initially supposed stay ...,0,0,0,0,138,70
1,--0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268.0,543.0,0.0,still love place took menu deeply saddened sho...,0,0,0,0,21,10
2,--1PwTt0AfyPG1pE7bB6Nw,pW9LvtC-hZ0PRAOUn7aZ2A,Alan is the best CPA that I know. He knows all...,5,0,8/15/2006,pW9LvtC-hZ0PRAOUn7aZ2A,834.0,1463.0,0.0,alan best know knows outs loop holes smart whi...,0,0,0,0,102,28
3,--87QVdPvgbDK3GeVxFRzg,92rtlWxS2aOngzkKhi-OxQ,My family has been using Korean Air for many m...,4,0,7/28/2012,92rtlWxS2aOngzkKhi-OxQ,43.0,158.0,0.0,family using korean many many years going asia...,0,0,0,0,135,62
4,--99ZJoVHP32iSd4i4nlZw,Egu4qSFR56EhGuTHZTaVjA,I hadn't been to Grisanti's since a pre-homeco...,3,0,1/28/2010,Egu4qSFR56EhGuTHZTaVjA,133.0,725.0,0.0,hadnt grisantis since prehomecoming dinner hig...,0,0,0,0,142,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,A18YXCahOIwmCJ1J8derdA,KYtrBL_Cp1CudlGC1XiiSA,I have to say that one of the best attributes ...,4,0,4/24/2010,KYtrBL_Cp1CudlGC1XiiSA,174.0,1022.0,0.0,best attributes place consistency around long ...,0,0,0,0,148,59
19996,a1a-7V09z2AfspiQeBiBCw,5Y5KbsI5buMcNh2hTFaRpA,Willowcreek is a decent option for vegetarians...,3,1,7/7/2011,5Y5KbsI5buMcNh2hTFaRpA,94.0,580.0,0.0,willowcreek decent option vegetariansvegans ha...,0,0,0,0,144,61
19997,a1AIpxn2OXX48s9RTmVr7g,wwNs_kMiVEn8V61uqP3ZzA,Rate location security and staff were better t...,4,3,7/18/2008,wwNs_kMiVEn8V61uqP3ZzA,5.0,12.0,0.0,rate location security staff better vast major...,0,0,0,1,153,63
19998,a1AoCyA9VE37XPa3eF5aQA,oClGUimXsTxxh4DvuKu26Q,this is the place I shop when in MT,4,0,7/31/2011,oClGUimXsTxxh4DvuKu26Q,21.0,105.0,0.0,place shop,0,0,0,0,9,2


**feature Extraction**

In [187]:
dateList=[]
for word in list_of_rows:
    if "Update" in str(word[5]):
        dateList.append(1)
    else:
        dateList.append(0)

extremeRating=[]
for rating in list(df["rating"]):
    if rating == 5 or rating == 1:
        extremeRating.append(1)
    else:
        extremeRating.append(0)




dict = {'UpdateOrNot': dateList,
       'NumberOfURLs': list(df["linksCount"]),
       'NumberOfnumerics': list(df["numericsCount"]),
      #'hashtagsCount': list(df["hashtagsCount"]),
      #'userMentionsCount': list(df["userMentionsCount"]),
       'extremeRating':extremeRating,
       'firstCount':list(df["firstCount"]),
       'wordCount':list(df['wCount']),
       'reviewCount':list(df['reviewCount']),
       'usefulCount':list(df['usefulCount']),
        'cosineSimilarty':cosineSim,
       'Filtered':list(df["Filtered"])
        }


df1 = pd.DataFrame(dict)
df1.to_csv('Data.csv',index=False)
df1.head()

,UpdateOrNot,NumberOfURLs,NumberOfnumerics,extremeRating,firstCount,wordCount,reviewCount,usefulCount,cosineSimilarty,Filtered
0,0,0,0,0,0.0,138,2.0,5,0.037917,0.0
1,1,0,0,0,268.0,21,543.0,2,0.031031,0.0
2,0,0,0,0,834.0,102,1463.0,0,0.021429,0.0
3,0,0,0,0,43.0,135,158.0,0,0.042153,0.0
4,0,0,0,0,133.0,142,725.0,0,0.042694,0.0


In [188]:
df1.info()
df1['Filtered'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   UpdateOrNot       20000 non-null  int64  
 1   NumberOfURLs      20000 non-null  int64  
 2   NumberOfnumerics  20000 non-null  int64  
 3   extremeRating     20000 non-null  int64  
 4   firstCount        20000 non-null  float64
 5   wordCount         20000 non-null  int64  
 6   reviewCount       20000 non-null  float64
 7   usefulCount       20000 non-null  object 
 8   cosineSimilarty   20000 non-null  float64
 9   Filtered          20000 non-null  float64
dtypes: float64(4), int64(5), object(1)
memory usage: 1.5+ MB


0.0    16671
1.0     3329
Name: Filtered, dtype: int64

In [189]:
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df1= df1.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna().astype(int)

In [190]:
X = df1.drop('Filtered', axis=1).astype(int, errors='ignore')
y = df1['Filtered'].astype(int, errors='ignore')

**handling imbalanced dataset by over sampling**

In [191]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
oversample = RandomOverSampler(sampling_strategy=1.0)
X, y = oversample.fit_resample(X, y)
#print(Counter(y_over))



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


**Split for train and test**

In [192]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.3)


**Decision Tree Classifier**

In [193]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Decision Tree:")
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("precision_score:",precision_score(y_test, y_pred)*100)
print("recall_score:",recall_score(y_test, y_pred)*100)
print("f1_score:",f1_score(y_test, y_pred)*100)


Decision Tree:
Accuracy: 89.20323902829152
precision_score: 83.5142118863049
recall_score: 97.52564876282437
f1_score: 89.97772828507794


**xgboost** **Classifier**

In [179]:
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

model = xgboost.XGBClassifier(subsample=0.85,learning_rate=0.3,max_depth= 95,max_features=1,min_samples_leaf=3,min_samples_split= 3,n_estimators= 1000)
kfold = StratifiedKFold(n_splits=3, random_state=None, shuffle=True)
for score in ["accuracy", "precision", "recall","f1"]:
  print( score," : ",cross_val_score(model, X,y,scoring=score, cv=kfold).mean()*100)

accuracy  :  92.12404774758562
precision  :  87.65313418021049
recall  :  97.75058484793954
f1  :  92.40827594329261
